# Modelo de DEEP LEARNING baseado na memória de curto prazo - LSTM

## 1. Bibliotecas

In [8]:
import math
import numpy                  as np
import pandas                 as pd
import yfinance               as yf

import matplotlib.pyplot      as plt

from datetime                 import datetime
from datetime                 import timedelta

from sklearn.preprocessing    import MinMaxScaler

from keras.models             import Sequential
from keras.layers             import Dense, LSTM

# pd.options.mode.chained_assigment = None

## Ativo de interesse

In [11]:
acao = 'MGLU3.SA'

inicio = '2014-12-31'
final  = '2023-05-26'

dados_acao = yf.download(acao, inicio, final)

dados_acao

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,0.243750,0.243750,0.231562,0.232812,0.215259,6323200
2015-01-05,0.235625,0.240625,0.229687,0.237187,0.219304,10326400
2015-01-06,0.238437,0.238437,0.233125,0.234062,0.216414,12572800
2015-01-07,0.237500,0.242187,0.234687,0.241875,0.223638,6454400
2015-01-08,0.239375,0.242187,0.237500,0.240000,0.221905,8393600
...,...,...,...,...,...,...
2023-05-19,3.580000,3.730000,3.530000,3.640000,3.640000,146332000
2023-05-22,3.670000,3.870000,3.660000,3.800000,3.800000,126815600
2023-05-23,3.830000,3.840000,3.640000,3.660000,3.660000,114830900


In [13]:
# Selecionaremos o valor de fechamento 'Close', ao invés da cotação ajustada 'Adj Close' (mais comunmente encontrada
# nos modelos de financças), pois precisamos de uma cotação real;

cotacao = dados_acao['Close'].to_numpy().reshape(-1,1) # reescalando os dados com o numpy
cotacao

array([[0.232812  ],
       [0.237187  ],
       [0.234062  ],
       ...,
       [3.66000009],
       [3.63000011],
       [3.75999999]])

In [15]:
# Selecionaremos 80% dos dados para o treinamento

tamanho_dados_treinamento = int(len(cotacao) * 0.8)
tamanho_dados_treinamento

1669

In [17]:
# Escalar os dados entre 0 e 1

escalador = MinMaxScaler(feature_range=(0, 1))

dados_entre_0_e_1_treinamento = escalador.fit_transform(cotacao[0:tamanho_dados_treinamento, :])

dados_entre_0_e_1_teste = escalador.transform(cotacao[tamanho_dados_treinamento: , :])

dados_entre_0_e_1 = list(dados_entre_0_e_1_treinamento.reshape(
    len(dados_entre_0_e_1_treinamento))) + list(dados_entre_0_e_1_teste.reshape(len(dados_entre_0_e_1_teste)))

dados_entre_0_e_1 = np.array(dados_entre_0_e_1).reshape(len(dados_entre_0_e_1), 1)

dados_entre_0_e_1

array([[0.00737532],
       [0.00753488],
       [0.00742091],
       ...,
       [0.13236661],
       [0.13127249],
       [0.13601366]])

In [18]:
dados_para_treinamento = dados_entre_0_e_1[0: tamanho_dados_treinamento, :]

# dados usados para gerar o resultado
treinamento_x = []

# cotação real
treinamento_y = []

for i in range(60, len(dados_para_treinamento)):
    
    # 60 últimos dias
    treinamento_x.append(dados_para_treinamento[i - 60: i, 0])
    
    # cotação real
    treinamento_y.append(dados_para_treinamento[i, 0])
    
    if i <= 61:
        print(treinamento_x)
        print(treinamento_y)


[array([0.00737532, 0.00753488, 0.00742091, 0.00770585, 0.00763747,
       0.00734115, 0.00743232, 0.00734115, 0.00720438, 0.00718159,
       0.00724997, 0.00720438, 0.00686247, 0.00691944, 0.00688527,
       0.00691944, 0.00686247, 0.00663453, 0.00641797, 0.0066915 ,
       0.00663453, 0.00668012, 0.006361  , 0.00628121, 0.00617865,
       0.00582532, 0.00575694, 0.00588233, 0.00549483, 0.0056088 ,
       0.00582532, 0.00585953, 0.00613306, 0.00612165, 0.00620145,
       0.00640659, 0.00622424, 0.0063838 , 0.00577974, 0.00581395,
       0.00558601, 0.00520988, 0.0051415 , 0.00495915, 0.00466283,
       0.00491356, 0.00486797, 0.00515292, 0.00487939, 0.00469704,
       0.00462865, 0.00457165, 0.00468562, 0.00470842, 0.00470842,
       0.00466283, 0.00458307, 0.00456027, 0.00452606, 0.00462865])]
[0.004685621274704243]
[array([0.00737532, 0.00753488, 0.00742091, 0.00770585, 0.00763747,
       0.00734115, 0.00743232, 0.00734115, 0.00720438, 0.00718159,
       0.00724997, 0.00720438, 0.00